# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvfile = 'Weather_DF.csv'
weather_df = pd.read_csv(csvfile)
weather_df.head(20)

,Unnamed: 0,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Elizabeth,-33.92,25.57,59.00,62,52,21.92,ZA,7/13/2020
1,1,Bredasdorp,-34.53,20.04,50.00,81,91,17.22,ZA,7/13/2020
2,2,East London,-33.02,27.91,69.62,39,3,15.95,ZA,7/13/2020
3,3,Huesca,42.14,-0.41,88.00,31,24,12.75,ES,7/13/2020
4,4,Hobart,-42.88,147.33,41.00,86,90,2.24,AU,7/13/2020
5,5,Mahébourg,-20.41,57.70,71.60,88,75,18.34,MU,7/13/2020
6,6,Port Alfred,-33.59,26.89,62.78,37,59,30.06,ZA,7/13/2020
7,7,Ngama,11.78,17.17,93.33,40,50,9.86,TD,7/13/2020
8,8,Albany,42.60,-73.97,82.00,53,10,3.78,US,7/13/2020
9,9,Beringovskiy,63.05,179.32,48.72,90,100,6.15,RU,7/13/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
latlonglocations = weather_df[["Lat", "Long"]]
weight = weather_df["Humidity"].astype(float)
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '2px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(latlonglocations, weights=weight, 
                                 dissipating=False, max_intensity = 100,
                                 point_radius=2)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='2px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
#My weather conditions: Max Temp </= 80, windspeed < 10, < 60 cloudiness, humidity < 40
# I am going to do a .loc with conjunctions to find my desired cities.
myweather_df = weather_df.loc[(weather_df['Max Temp'] <= 80)\
                              & (weather_df['Wind Speed'] <= 10)\
                              & (weather_df['Cloudiness'] < 60)\
                             & (weather_df['Humidity']< 35)]
# Drop null/NA values and display DF
myweather_df.dropna()
myweather_df = myweather_df.drop(columns=['Unnamed: 0'])
myweather_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,Yumen,40.28,97.20,74.16,32,54,6.33,CN,7/13/2020
185,Green River,41.53,-109.47,71.60,28,1,8.05,US,7/13/2020
196,Valparaíso,-33.04,-71.63,68.00,34,0,8.05,CL,7/13/2020
266,Ondangwa,-17.92,15.95,73.78,18,0,7.16,NaN,7/13/2020
291,Kang,-23.68,22.79,62.28,25,0,7.45,BW,7/13/2020
375,Huambo,-12.78,15.74,77.00,20,2,6.93,AO,7/13/2020
473,Kangāvar,34.50,47.97,76.82,19,0,6.22,IR,7/13/2020
531,Eenhana,-17.47,16.33,76.32,16,0,5.19,NaN,7/13/2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
#hotel_df = myweather_df.insert(["Hotel Name"])
hotel_df = myweather_df.loc[:,["City","Country", "Lat", "Long"]]
hotel_df['Hotel Name']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lng']=""
hotel_df

,City,Country,Lat,Long,Hotel Name,Hotel Lat,Hotel Lng
94,Yumen,CN,40.28,97.20,,,
185,Green River,US,41.53,-109.47,,,
196,Valparaíso,CL,-33.04,-71.63,,,
266,Ondangwa,NaN,-17.92,15.95,,,
291,Kang,BW,-23.68,22.79,,,
375,Huambo,AO,-12.78,15.74,,,
473,Kangāvar,IR,34.50,47.97,,,
531,Eenhana,NaN,-17.47,16.33,,,


In [25]:
# Set up Parameter variables
target_search = "hotel"
target_radius = 5000
target_type = "lodging"
# set up a Parameters dictionary
params = {"keyword": target_search,"radius": target_radius,"type": target_type,"key": g_key}
# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()

In [26]:
# find the closest hotel of each type to coordinates
# use iterrows to iterate through pandas dataframe
#latlist = []
#longlist = []

print('''
---------------------
Beginning your search
---------------------
''')
for index, row in hotel_df.iterrows():
    # Retrieve hotels but I have to go by the lat and long
    lat = row["Lat"]
    long = row["Long"]
    name = row["City"]
    # add keyword to params dict
    params["location"] = (f"{lat},{long}")
    # assemble url and make API request
    print(f"Retrieving Results for {name}.")
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    try:
        print(f"The closest hotel to {name} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        hotel_df.loc[index, "Hotel Lat"] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, "Hotel Lng"] = results[0]['geometry']['location']['lng']
        
    except(KeyError, IndexError):
        print(f"Could not find a hotel for {name}... skipping to next vacation spot.")
        hotel_df.loc[index, "Hotel Name"] = "Not Found"
        hotel_df.loc[index, "Hotel Lat"] = hotel_df.loc[index, "Lat"]
        hotel_df.loc[index, "Hotel Lng"] = hotel_df.loc[index, "Long"]
    print("------------")
    
print('''
------------------------
The search has concluded
------------------------
''')


---------------------
Beginning your search
---------------------

Retrieving Results for Yumen.
Could not find a hotel for Yumen... skipping to next vacation spot.
------------
Retrieving Results for Green River.
The closest hotel to Green River is Hampton Inn & Suites Green River
------------
Retrieving Results for Valparaíso.
The closest hotel to Valparaíso is Hotel Palacio Astoreca
------------
Retrieving Results for Ondangwa.
The closest hotel to Ondangwa is Palm View Hotel
------------
Retrieving Results for Kang.
The closest hotel to Kang is Phofu Eco Safari Lodge
------------
Retrieving Results for Huambo.
The closest hotel to Huambo is Hotel Ekuikui I
------------
Retrieving Results for Kangāvar.
Could not find a hotel for Kangāvar... skipping to next vacation spot.
------------
Retrieving Results for Eenhana.
The closest hotel to Eenhana is Hotel Litu
------------

------------------------
The search has concluded
------------------------



In [28]:
hotel_df['Hotel Lat'] = hotel_df['Hotel Lat'].astype(float)
hotel_df['Hotel Lng'] = hotel_df['Hotel Lng'].astype(float)
hotel_df

,City,Country,Lat,Long,Hotel Name,Hotel Lat,Hotel Lng
94,Yumen,CN,40.28,97.20,Not Found,40.280000,97.200000
185,Green River,US,41.53,-109.47,Hampton Inn & Suites Green River,41.540598,-109.472282
196,Valparaíso,CL,-33.04,-71.63,Hotel Palacio Astoreca,-33.040612,-71.628483
266,Ondangwa,NaN,-17.92,15.95,Palm View Hotel,-17.930118,15.989866
291,Kang,BW,-23.68,22.79,Phofu Eco Safari Lodge,-23.680756,22.766045
375,Huambo,AO,-12.78,15.74,Hotel Ekuikui I,-12.778276,15.734130
473,Kangāvar,IR,34.50,47.97,Not Found,34.500000,47.970000
531,Eenhana,NaN,-17.47,16.33,Hotel Litu,-17.487921,16.334376


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [30]:
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '2px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
gmaps.figure(layout=figure_layout)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='2px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…